# Dublin Bike Sharing

Ronan Downes | [Github](https://github.com/ronandownes/dublinbikes-CA1) | November 2022 
***


# Loads Bike and Dock dataset 2019
Downloads  quarterly  Dublin Bike csv files directly from the web.  The datasets are freely provided by the [Smart Dublin](https://data.smartdublin.ie/dataset/dublinbikes-api).  
***

In [1]:
#Makes a data directory  and imports libraries

import pandas as pd
import numpy as np
import warnings
import time
import datetime as dt
warnings.filterwarnings('ignore')
import matplotlib.pyplot as plt
import urllib
import folium
#import sklearn
import seaborn as sns
import os
#from sklearn.model_selection import train_test_split
from matplotlib import pyplot as plt
#from sklearn.cluster import KMeans
# from matplotlib import inline
from ipywidgets import interact, interact_manual


if not os.path.exists("data"):
    os.makedirs("data")

## Loading Bike and Dock occupancy data

In [2]:

 ### Defines a list of tuples with ordered pairs of filenames and retrieval urls. 
    
dataframes = [
    (
        "data/dublinbikes_20190101_20190401.csv",
        "https://data.smartdublin.ie/dataset/33ec9fe2-4957-4e9a-ab55-c5e917c7a9ab/resource/538165d7-535e-4e1d-909a-1c1bfae901c5/download/dublinbikes_20190101_20190401.csv",
    ),
    (
        "data/dublinbikes_20190401_20190701.csv",
        "https://data.smartdublin.ie/dataset/33ec9fe2-4957-4e9a-ab55-c5e917c7a9ab/resource/76fdda3d-d8be-441b-92dd-0ee36d9c5316/download/dublinbikes_20190401_20190701.csv",
    ),
    (
        "data/dublinbikes_20190701_20191001.csv",
        "https://data.smartdublin.ie/dataset/33ec9fe2-4957-4e9a-ab55-c5e917c7a9ab/resource/305d39ac-b6a0-4216-a535-0ae2ddf59819/download/dublinbikes_20190701_20191001.csv",
    ),
    (
        "data/dublinbikes_20191001_20200101.csv",
        "https://data.smartdublin.ie/dataset/33ec9fe2-4957-4e9a-ab55-c5e917c7a9ab/resource/5d23332e-4f49-4c41-b6a0-bffb77b33d64/download/dublinbikes_20191001_20200101.csv",
    ),
        (
        "data/dublinbikes_20200101_20200401.csv",
        "https://data.smartdublin.ie/dataset/33ec9fe2-4957-4e9a-ab55-c5e917c7a9ab/resource/aab12e7d-547f-463a-86b1-e22002884587/download/dublinbikes_20200101_20200401.csv",
    ),
]

## Define the names and locations of the consolidated Bike and Dock availability csv and the weather csv.

In [3]:
bikes_df = "data/bikes.csv"
weather_df = "data/weather.csv"

In [4]:
# Retrieve data functions for historical weather data from the Irish Meteorological Society websites 
for item in dataframes:
    if os.path.exists(item[0]):
        continue
    print(f"Downloading {item[0]} from {item[1]}")
    urllib.request.urlretrieve(item[1], item[0])
    
def retrieve_weather():
    weather_url = "http://cli.fusio.net/cli/climate_data/webdata/hly175.csv"
    urllib.request.urlretrieve(weather_url, weather_df)
    
    #Conditionally calls  retrieval functions if CSVs are not in place in the"data" directory
    # The file is called HLY175 because data points are recorded hourly 
    # and Phoenix park is station 175
    
if not os.path.exists(weather_df):
    retrieve_weather()

In [5]:
#Conditionally calls  weather retrieval function
if not os.path.exists(weather_df):
    retrieve_weather()

In [6]:
# Read and concatenate quarterly bike data  
# Read  Phoinex Park (station 175) weather data from the Irish Meteorological Society website on an hourly basis. Original file is hly175.csv
bikes_df = "data/bikes.csv"
weather_df = pd.read_csv("data/weather.csv", skiprows=15)
bikes_df = pd.concat([pd.read_csv(item[0]) for item in dataframes])

if not os.path.exists("data/bikes.csv"):
    bikes_df.to_csv("data/bikes.csv", index=False)
    
    
if not os.path.exists("data/weather.csv"):
    weather_df.to_csv("data/weather.csv", index=False)    

In [7]:
bikes_df.tail(5)

,STATION ID,TIME,LAST UPDATED,NAME,BIKE STANDS,AVAILABLE BIKE STANDS,AVAILABLE BIKES,STATUS,ADDRESS,LATITUDE,LONGITUDE
2228273,117,2020-04-01 23:35:02,2020-04-01 23:32:46,HANOVER QUAY EAST,40,37,3,Open,Hanover Quay East,53.343655,-6.231755
2228274,117,2020-04-01 23:40:02,2020-04-01 23:32:46,HANOVER QUAY EAST,40,37,3,Open,Hanover Quay East,53.343655,-6.231755
2228275,117,2020-04-01 23:45:02,2020-04-01 23:42:54,HANOVER QUAY EAST,40,37,3,Open,Hanover Quay East,53.343655,-6.231755
2228276,117,2020-04-01 23:50:03,2020-04-01 23:42:54,HANOVER QUAY EAST,40,37,3,Open,Hanover Quay East,53.343655,-6.231755
2228277,117,2020-04-01 23:55:02,2020-04-01 23:53:01,HANOVER QUAY EAST,40,37,3,Open,Hanover Quay East,53.343655,-6.231755


In [8]:
# close both dataframes

bikes_df = None
weather_df = None